## Setup working enviroment 

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install kaggle ijson pyTigerGraph

Processing /Users/ybenami/Library/Caches/pip/wheels/79/40/57/49eee0c279b56aa3e06d10708dcfb53142240861b122303f2b/kaggle-1.5.6-py3-none-any.whl
Processing /Users/ybenami/Library/Caches/pip/wheels/ac/7c/93/20f4c74a96b2e48d7dd964d497a577f2f95cec10efeb6335cd/ijson-3.1.post0-py3-none-any.whl
Processing /Users/ybenami/Library/Caches/pip/wheels/87/da/a2/f1c8357abfe6638e2218ca1a5d376d7e628a0d6e018abbd6bf/pyTigerGraph-0.0.6.2-py3-none-any.whl
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
Processing /Users/ybenami/Library/Caches/pip/wheels/91/4d/4f/e740a68c215791688c46c4d6251770a570e8dfea91af1acb5c/python_slugify-4.0.1-py2.py3-none-any.whl
Processing /Users/ybenami/Library/Caches/pip/wheels/73/f4/8a/14292ff2db9a42b0c8c0cb75d059a3251af273964d05409d3a/validators-0.15.0-py3-none-any.whl
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.9
    Uninstalling urllib3-1.25.9:
      Successfully uninsta

In [3]:
import sys
!{sys.executable} -m pip install -e ..

Obtaining file:///Users/ybenami/EasyAsPie.ai/autoTigerGraph
  Running setup.py develop for autoTigerGraph


In [2]:
import autoTigerGraph

## Launch and TigerGraph Developer instance on AWS 
Use the following command in a shell to connect to it 

`ssh -L14240:localhost:14240 -L 8888:localhost:8888 -i your_aws_key.pem ubuntu@your-server-ip`

Check the connection by clicking on the link below. It should open TigerStudio in a browser window

In [4]:
server = 'http://localhost'
print(server+':14240')

http://localhost:14240


In [5]:
import pyTigerGraph as tg

conn = tg.TigerGraphConnection(host=server, graphname='yelp')
shell = tg.Gsql(conn , certNeeded=False)

print(shell.gsql('ls', options=[]))

Trying version: v2_6_0
Connecting to localhost:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
---- Global vertices, edges, and all graphs
Vertex Types: 
  - VERTEX Business(PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX User(PRIMARY_ID user_id STRING, name STRING, review_count INT, yelping_since STRING, useful INT, funny INT, cool INT, elite STRING, friends STRING, fans INT, average_stars DOUBLE, compliment_hot INT, compliment_more INT, compliment_profile INT, compliment_cute INT, compliment_list INT, compliment_note INT, compliment_plain INT, compliment_cool INT, compliment_funny INT, compliment_writer INT, compliment_photos INT) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX Review(PRIMARY_ID review_id STRING, user_id S

## Load Yelp Dataset from Kaggle

see https://www.kaggle.com/docs/api

In [6]:
from ipywidgets import FileUpload
upload = FileUpload()
upload

FileUpload(value={}, description='Upload')

In [10]:
import os

!mkdir ~/.kaggle
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w+b") as i:
    i.write(upload.data[0])
    
!chmod 600 ~/.kaggle/kaggle.json

mkdir: /Users/ybenami/.kaggle: File exists


In [11]:
!kaggle datasets download yelp-dataset/yelp-dataset
!unzip yelp-dataset

100%|██████████████████████████████████████| 4.48G/4.48G [05:34<00:00, 12.4MB/s]
100%|██████████████████████████████████████| 4.48G/4.48G [05:34<00:00, 14.4MB/s]
Archive:  yelp-dataset.zip
  inflating: Dataset_Agreement.pdf   
  inflating: yelp_academic_dataset_business.json  
  inflating: yelp_academic_dataset_checkin.json  
  inflating: yelp_academic_dataset_review.json  
  inflating: yelp_academic_dataset_tip.json  
  inflating: yelp_academic_dataset_user.json  


In [12]:
ls -l

total 29854992
-rw-r--r--  1 ybenami  ybenami       41776 Mar 26 01:18 Dataset_Agreement.pdf
-rw-r--r--@ 1 ybenami  ybenami       20582 Jun 30 18:08 YelpGraph.ipynb
-rw-r--r--  1 ybenami  ybenami  4809540040 Jun 30 18:07 yelp-dataset.zip
-rw-r--r--  1 ybenami  ybenami   152898689 Mar 26 01:18 yelp_academic_dataset_business.json
-rw-r--r--  1 ybenami  ybenami   449663480 Mar 26 01:18 yelp_academic_dataset_checkin.json
-rw-r--r--  1 ybenami  ybenami  6325565224 Mar 26 01:19 yelp_academic_dataset_review.json
-rw-r--r--  1 ybenami  ybenami   263489322 Mar 26 01:31 yelp_academic_dataset_tip.json
-rw-r--r--  1 ybenami  ybenami  3268069927 Mar 26 01:32 yelp_academic_dataset_user.json


In [10]:
!rm yelp-dataset.zip

In [19]:
import autoTigerGraph as atg
import importlib

importlib.reload(atg)

<module 'autoTigerGraph' from '/Users/ybenami/EasyAsPie.ai/autoTigerGraph/autoTigerGraph/__init__.py'>

## Create schema and load data

In [20]:
filename = 'yelp_academic_dataset_business.json'
business, fields = atg.get_first(filename)
business, fields

({'business_id': 'f9NumwFMBDn751xgFiRbNA',
  'name': 'The Range At Lake Norman',
  'address': '10913 Bailey Rd',
  'city': 'Cornelius',
  'state': 'NC',
  'postal_code': '28031',
  'latitude': 35.4627242,
  'longitude': -80.8526119,
  'stars': 3.5,
  'review_count': 36,
  'is_open': 1,
  'attributes': {'BusinessAcceptsCreditCards': 'True',
   'BikeParking': 'True',
   'GoodForKids': 'False',
   'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
   'ByAppointmentOnly': 'False',
   'RestaurantsPriceRange2': '3'},
  'categories': 'Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping',
  'hours': {'Monday': '10:0-18:0',
   'Tuesday': '11:0-20:0',
   'Wednesday': '10:0-18:0',
   'Thursday': '11:0-20:0',
   'Friday': '11:0-20:0',
   'Saturday': '11:0-20:0',
   'Sunday': '13:0-18:0'}},
 [(0, 'business_id', 'str'),
  (1, 'name', 'str'),
  (2, 'address', 'str'),
  (3, 'city', 'str'),
  (4, 'state', 'str'),
  (5, 'postal_code', 'str'),
  (6,

In [21]:
print(shell.gsql('create graph yelp (*)', options=[]))

Trying version: v2_6_0
Connecting to localhost:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
Semantic Check Fails: The graph name conflicts with another type or existing graph names! Please use a different name.
The graph yelp could not be created!



In [22]:
atg.guess_vertex(json_object=business, vertex_name='Business')

'CREATE VERTEX Business (PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING)'

In [23]:
filename = 'yelp_academic_dataset_business.json'
vertex_name = 'Business'
primary_id = 'business_id'

json_object, _ = atg.get_first(filename)
atg.create_vertex(shell=shell, json_object=json_object, vertex_name=vertex_name, graph_name='yelp')


Trying version: v2_6_0
Connecting to localhost:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
The graph yelp is dropped.
The vertex type Business is dropped.
The vertex type Business is created.
The graph yelp is created.
---- Graph yelp
Vertex Types: 
  - VERTEX User(PRIMARY_ID user_id STRING, name STRING, review_count INT, yelping_since STRING, useful INT, funny INT, cool INT, elite STRING, friends STRING, fans INT, average_stars DOUBLE, compliment_hot INT, compliment_more INT, compliment_profile INT, compliment_cute INT, compliment_list INT, compliment_note INT, compliment_plain INT, compliment_cool INT, compliment_funny INT, compliment_writer INT, compliment_photos INT) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX Review(PRIMARY_ID review_id STRING, user_id STRING, business_id STRING, stars DOUBLE, useful INT, funny INT, cool INT, text STRING, date_time DATETIME) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX Business(PRIMARY_ID business_id STRING, n

In [ ]:
atg.upsert_json(filename=filename, conn=conn, vertex_name=vertex_name, 
                primary_id=primary_id, n=10000)

In [16]:
filename = 'yelp_academic_dataset_user.json'
vertex_name = 'User'
primary_id = 'user_id'

json_object, _ = atg.get_first(filename)
atg.create_vertex(shell=shell, json_object=json_object, vertex_name=vertex_name, graph_name='yelp')
atg.upsert_json(filename=filename, conn=conn, vertex_name=vertex_name, 
                primary_id=primary_id, n=10000)

Trying version: v2_6_0
Connecting to localhost:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
The graph yelp is dropped.
The vertex type User could not be found.
The vertex type User is created.
The graph yelp is created.
---- Graph yelp
Vertex Types: 
  - VERTEX Business(PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX User(PRIMARY_ID user_id STRING, name STRING, review_count INT, yelping_since STRING, useful INT, funny INT, cool INT, elite STRING, friends STRING, fans INT, average_stars DOUBLE, compliment_hot INT, compliment_more INT, compliment_profile INT, compliment_cute INT, compliment_list INT, compliment_note INT, compliment_plain INT, compliment_cool INT, compliment_funny INT, compliment_writer INT, compliment_pho

1968703

In [17]:
filename = 'yelp_academic_dataset_review.json'
vertex_name = 'Review'
primary_id = 'review_id'

json_object, _ = atg.get_first(filename)
atg.create_vertex(shell=shell, json_object=json_object, vertex_name=vertex_name, graph_name='yelp')

Trying version: v2_6_0
Connecting to localhost:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
The graph yelp is dropped.
The vertex type Review could not be found.
The vertex type Review is created.
The graph yelp is created.
---- Graph yelp
Vertex Types: 
  - VERTEX Business(PRIMARY_ID business_id STRING, name STRING, address STRING, city STRING, state STRING, postal_code STRING, latitude DOUBLE, longitude DOUBLE, stars DOUBLE, review_count INT, is_open INT, attributes STRING, categories STRING, hours STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX User(PRIMARY_ID user_id STRING, name STRING, review_count INT, yelping_since STRING, useful INT, funny INT, cool INT, elite STRING, friends STRING, fans INT, average_stars DOUBLE, compliment_hot INT, compliment_more INT, compliment_profile INT, compliment_cute INT, compliment_list INT, compliment_note INT, compliment_plain INT, compliment_cool INT, compliment_funny INT, compliment_writer INT, compliment

In [18]:
atg.upsert_json(filename=filename, conn=conn, vertex_name=vertex_name, 
                primary_id=primary_id, n=10000)

8021122